# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$.
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки.
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вершину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии.

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [37]:
def entropy(y : pd.Series):

    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))

    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер.

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения узлов дерева (используйте, например, `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признаков и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три разных критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики доля правильных ответов (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [171]:
from numbers import Number
from sklearn.datasets import load_iris

def split_df(df : pd.DataFrame, j : int, t : Number):
    return df[df[df.columns[j]] < t], df[df[df.columns[j]] >= t]

def calc_IG(R_v : pd.DataFrame, R_left : pd.DataFrame, R_right : pd.DataFrame):
    return entropy(R_v[R_v.columns[-1]]) - (
        R_left.shape[0] / R_v.shape[0] * entropy(R_left[R_left.columns[-1]]) + 
        R_right.shape[0] / R_v.shape[0] * entropy(R_right[R_right.columns[-1]])
    )

def get_best_split(R_v : pd.DataFrame):
    best_j, best_t, max_IG = 0, 0.0, 0.0
    for j in range(len(R_v.columns) - 1):
        for t in range(len(R_v)):
            left, right = split_df(R_v, j, t)
            IG = calc_IG(R_v, left, right)
            if IG > max_IG:
                best_j = j
                best_t = t
                max_IG = IG
    left, right = split_df(R_v, best_j, best_t)
    return {'col': best_j, 't': best_t, 'left': left, 'right': right}

def is_pure(R_v : pd.DataFrame):
    return R_v[R_v.columns[-1]].value_counts().size == 1

def to_leaf(df : pd.DataFrame):
    return df[df.columns[-1]].value_counts().index[0]

def split_node(depth, parent, max_depth, min_obj):
    left = parent['left']
    right = parent['right']

    if left.empty or right.empty:
        parent['left'] = parent['right'] = to_leaf(pd.concat([left, right]))
        return

    if depth >= max_depth:
        parent['left'] = to_leaf(left)
        parent['right'] = to_leaf(right)
        return

    if is_pure(left) or len(left) <= min_obj:
        parent['left'] = to_leaf(left)
    else:
        parent['left'] = get_best_split(left)
        split_node(depth + 1, parent['left'], max_depth, min_obj)

    if is_pure(right) or len(right) <= min_obj:
        parent['right'] = to_leaf(right)
    else:
        parent['right'] = get_best_split(right)
        split_node(depth + 1, parent['right'], max_depth, min_obj)

def grow_tree(df, max_depth, min_obj):
    root = get_best_split(df)
    split_node(1, root, max_depth, min_obj)
    return root

def predict_row(node, df_row : pd.Series):
    if df_row.values[node['col']] < node['t']:
        if isinstance(node['left'], dict):
            return predict_row(node['left'], df_row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict_row(node['right'], df_row)
        else:
            return node['right']

def predict(root, df):
    y_pred = []
    for i, row in df.iterrows():
        y_pred.append(predict_row(root, row))
    return y_pred


In [44]:
df = load_iris()
data_full = pd.DataFrame(data=np.c_[df['data'], df['target']], columns=df['feature_names'] + ['target'])
data_full

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [186]:
from sklearn.metrics import accuracy_score

seed = random.randint(1, 1000)
df_train = data_full.sample(100, random_state=seed)
df_test = data_full.drop(df_train.index)
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

root = grow_tree(df_train, 5, 5)
accuracy_score(df_test[df_test.columns[-1]], predict(root, df_test))

0.94

##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес".

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [193]:
%pip install category_encoders

   ---------------------------------------- 0.0/81.8 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/81.8 kB ? eta -:--:--
   --------------- ------------------------ 30.7/81.8 kB 435.7 kB/s eta 0:00:01
   -------------------- ------------------- 41.0/81.8 kB 487.6 kB/s eta 0:00:01
   ---------------------------------------- 81.8/81.8 kB 761.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [222]:
data = pd.read_csv('tables/churn.csv')
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [223]:
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler

data_target = data['Exited']
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)
data['Gender'].replace(['Male', 'Female'], [1, 0], inplace=True)
data['Geography'] = TargetEncoder().fit_transform(data['Geography'], data_target)
data[['CreditScore', 'Age', 'Balance', 'EstimatedSalary']] = StandardScaler().fit_transform(data[['CreditScore', 'Age', 'Balance', 'EstimatedSalary']])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,-0.326221,0.161548,0,0.293517,2,-1.225848,1,1,1,0.021886,1
1,-0.440036,0.166734,0,0.198164,1,0.117350,1,0,1,0.216534,0
2,-1.536794,0.161548,0,0.293517,8,1.333053,3,1,0,0.240687,1
3,0.501521,0.161548,0,0.007457,1,-1.225848,2,0,0,-0.108918,0
4,2.063884,0.166734,0,0.388871,2,0.785728,1,1,1,-0.365276,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,1.246488,0.161548,1,0.007457,5,-1.225848,2,1,0,-0.066419,0
9996,-1.391939,0.161548,1,-0.373958,10,-0.306379,1,1,1,0.027988,0
9997,0.604988,0.161548,0,-0.278604,7,-1.225848,1,0,1,-1.008643,1
9998,1.256835,0.324432,1,0.293517,3,-0.022608,2,1,0,-0.125231,1


In [323]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

def grow_forest(X, y, n_trees=100, max_depth=20, min_samples=5):
    m = len(X)
    k = int(np.sqrt(len(X.columns)))
    trees = []
    for _ in range(n_trees):
        seed = random.randint(1, 1000)
        X_sample = X.sample(n=m, random_state=seed, replace=True)
        y_sample = y.sample(n=m, random_state=seed, replace=True)
        tree = DecisionTreeClassifier(max_depth=max_depth, max_features=k, min_samples_leaf=min_samples)
        tree.fit(X_sample, y_sample)
        trees.append(tree)
    return trees

def predict_forest(forest, X):
    votes = np.array([tree.predict(X) for tree in forest]).swapaxes(0, 1)
    return np.array([np.bincount(row).argmax() for row in votes])

def get_best_hyperparams(X_train, y_train, X_test, y_test):
    best_tree_cnt = best_depth = best_samples = 0
    best_score = -1
    for tree_cnt in [100, 200, 300]:
        for depth in range(1, 52, 10):
            for samples_cnt in range(1, 4):
                forest = grow_forest(X_train, y_train, tree_cnt, depth, samples_cnt)
                y_pred = predict_forest(forest, X_test)
                score = accuracy_score(y_test, y_pred)
                if (score > best_score):
                    best_tree_cnt, best_depth, best_samples = tree_cnt, depth, samples_cnt
    return best_tree_cnt, best_depth, best_samples

def feature_importances(forest):
    votes = []
    for tree in forest:
        votes.append(tree.feature_importances_)
    votes = pd.DataFrame(votes)
    return votes.mean()

In [313]:
data_X = data.drop('Exited', axis=1)
data_y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, random_state=17)

forest = grow_forest(X_train, y_train)

In [314]:
y_pred = predict_forest(forest, X_test)
accuracy_score(y_test, y_pred)

0.8592

In [ ]:
best_params = get_best_hyperparams(X_train, y_train, X_test, y_test)
best_forest = grow_forest(X_train, y_train, *best_params)

In [322]:
pd.DataFrame(data = np.c_[X_test.columns, feature_importances(best_forest)], columns = ['Feature', 'Importance']).sort_values(by='Importance', ascending=False)

,Feature,Importance
3,Age,0.264004
6,NumOfProducts,0.147922
5,Balance,0.134915
9,EstimatedSalary,0.124233
0,CreditScore,0.124218
4,Tenure,0.068144
8,IsActiveMember,0.059625
1,Geography,0.043623
2,Gender,0.018341
7,HasCrCard,0.014974


In [324]:
best_params

(300, 51, 3)

Наиболее важным признаком оказался возраст клиента, наименее важными - пол клиента и наличие у него кредитной карты